In [221]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

In [222]:
llm = OllamaLLM(model="gemma3:1b")

In [223]:
class QAState(TypedDict):
    question: str
    answer: str

In [224]:
graph = StateGraph(QAState)

In [225]:
def qa_function(state: QAState) -> QAState:
    user_query = state["question"]
    Prompt = PromptTemplate(
        template="Answer the question: {user_query}",
        input_variables=["user_query"]
    )

    formatted_prompt = Prompt.format(user_query=user_query)
    response = llm.invoke(formatted_prompt)

    # Make sure to get response content correctly
    state["answer"] = response.content if hasattr(response, "content") else response
    return state
    return state

In [226]:
graph.add_node("qa_function", qa_function)

In [227]:
graph.add_edge(START , "qa_function")
graph.add_edge("qa_function", END)

In [228]:
workflow = graph.compile()

In [229]:
user_query = {
    'question': "What is the capital of England?"
}
final_state = workflow.invoke(user_query)

In [230]:
print(final_state)

{'question': 'What is the capital of England?', 'answer': 'The capital of England is **London**. \n\nDo you want to know more about London?'}
